<center><img src="../image/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 10.机器人二维码识别控制运动

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：10.机器人二维码识别控制运动.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：机器人二维码识别控制运动<br>

## 为了可以打开摄像头，这里关闭摄像头所有进程

In [1]:
# 载入必要的库
import cv2
import numpy as np
from pyzbar.pyzbar import decode
from PIL import Image, ImageDraw, ImageFont
from LOBOROBOT import LOBOROBOT              # 载入机器人库
import sys 
import time

In [2]:
if sys.getdefaultencoding() != 'utf-8':
    reload(sys)
    sys.setdefaultencoding('utf-8')

In [3]:
clbrobot = LOBOROBOT()  # 实例化机器人对象
# Configure min and max servo pulse lengths
servo_min = 150  # Min pulse length out of 4096
servo_max = 600  # Max pulse length out of 4096

In [4]:
# 舵机调零
pan =  90
tilt = -10
# 频率设置为50hz，适用于舵机系统。
clbrobot.set_servo_angle(10,pan)  # 底座舵机 90 
clbrobot.set_servo_angle(9,tilt)  # 顶部舵机 30
time.sleep(0.5)

In [5]:
# 智能机器人运动状态
enStop = 0
enRun = 1
enBack = 2
enLeft = 3
enRight = 4
car_State = 0
symbolPos = []

last_qr_data = 'no qrcode'
qr_data = ' '

## 二维码识别框绘制

In [6]:
def draw_rect(img, pos, color, width):
    cv2.line(img, pos[0], pos[1], color, width)
    cv2.line(img, pos[0], pos[3], color, width)
    cv2.line(img, pos[2], pos[1], color, width)
    cv2.line(img, pos[2], pos[3], color, width)

In [7]:
def qr_scan_decode(img):
    global qr_data
    global last_qr_data
    
    for barcode in decode(img):
        (x, y, w, h) = barcode.rect
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)
        barcodeData = barcode.data.decode("utf-8")
        barcodeType = barcode.type
        if not len(barcodeData) == 0:
            qr_data = str(barcodeData)
            print(qr_data)
        else:
            qr_data = 'no qrcode'
            
    if last_qr_data != qr_data:       
        command_resolve(qr_data)
    last_qr_data = qr_data

In [8]:
def command_resolve(command):
    last_car_state = 0
    if command.find("Run",0,len(command)) != -1 :
        command.zfill(len(command))
        car_state = 1
    elif command.find("Back",0,len(command)) != -1 :
        command.zfill(len(command))
        car_state = 2
    elif command.find("Left",0,len(command)) != -1 :
        command.zfill(len(command))
        car_state = 3
    elif command.find("Right",0,len(command)) != -1:
        command.zfill(len(command))
        car_state = 4
    elif command.find("Stop",0,len(command)) != -1:            
        command.zfill(len(command))
        car_state = 5
    else:
        car_state = 5
        command.zfill(len(command))
        
    if last_car_state != car_state :
        car_control(car_state)
    last_car_state = car_state

In [9]:
def car_control(car_state):
    print(car_state)
    if car_state == enRun:
        content='机器人前进'
        clbrobot.t_up(30,1)         # 前进
    elif car_state == enBack :
        content='机器人后退'
        clbrobot.t_down(30,1)      # 后退
    elif car_state == enLeft :
        content='机器人左转'      
        clbrobot.turnLeft(30,1)    # 左转
    elif car_state == enRight :
        content='机器人右转'
        clbrobot.turnRight(30,1)   # 右转
    elif car_state == enStop:
        content='机器人停止'
        clbrobot.t_stop(0)        # 停止
    else:
        content='机器人停止'
        clbrobot.t_stop(0)       # 停止


In [10]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [11]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])
    
frame_img = widgets.Image(format='jpeg', width=320, height=240)
display(frame_img)

Image(value=b'', format='jpeg', height='240', width='320')

In [12]:
import libcamera
from picamera2 import Picamera2

picamera = Picamera2()
config = picamera.create_preview_configuration(main={"format": 'RGB888', "size": (320, 240)},
                                               raw={"format": "SRGGB12", "size": (1920, 1080)})
config["transform"] = libcamera.Transform(hflip=0, vflip=0)
picamera.configure(config)
picamera.start()

def Video_display():
    # 设置Camera
    while(True):
        frame = picamera.capture_array()
        frame=cv2.flip(frame, 0) 
        qr_scan_decode(frame)
        frame_img.value = bgr8_to_jpeg(frame)

[1:55:52.548300301] [3905]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[1:55:52.557008561] [4005]  INFO RPI pisp.cpp:653 libpisp version v1.0.2 fa44a258644a 22-11-2023 (21:59:22)
[1:55:52.566888497] [4005]  INFO RPI pisp.cpp:1112 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media2 and ISP device /dev/media0 using PiSP variant BCM2712_C0
[1:55:52.568919331] [3905]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[1:55:52.577370205] [4008]  INFO RPI pisp.cpp:653 libpisp version v1.0.2 fa44a258644a 22-11-2023 (21:59:22)
[1:55:52.586991791] [4008]  INFO RPI pisp.cpp:1112 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media2 and ISP device /dev/media0 using PiSP variant BCM2712_C0
[1:55:52.589552454] [3905]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format Y16 
[1:55:52.589589028] [3905]  WARN V4L2 v4l2_pixelformat.cpp:338 Unsupported V4L2 pixel format RGB6
[1:55:

In [13]:
# 开始线程
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

/tmp/ipykernel_3905/466567028.py:3: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  t.setDaemon(True)


5


In [ ]:
# 结束线程
stop_thread(t)

In [ ]:
clbrobot.t_stop(0)